In [20]:
import os
import sys
import json

from logging import error, warning


CLASS_MAP = {
    'Machine-translated / generated texts': 'MT/Gen',
    'Description with intent to sell': 'D-Sell',
    'Personal blog': 'B-Personal',
    'Description of a thing': 'D-Thing',
    'News reports / News blogs': 'News',
    'News reports / news blogs': 'News',    # typo fix
    'How-to/instructions': 'How-to',
    'Religious blogs/sermons': 'Religious',
    'Personal opinion blogs': 'B-Personal-Opinion',
    'Discussion forums': 'Forums',
    'Reviews': 'Reviews',
    'Encyclopedia articles': 'A-Encyclopedia',
    'Community blogs': 'B-Community',
    'Community blog': 'B-Community',    # typo fix
    'Sports reports': 'Sports',
    'News+Opinion blogs / Editorials': 'Editorials',
    'Description of a person': 'D-Person',
    'Information blogs': 'B-Information',
    'Online article': 'A-Online',
    'Research articles': 'A-Research',
    'Historical article': 'A-Historical',
    'Question-Answer forums': 'QA-forums',
    'Advice': 'Advice',
    'Travel blog': 'B-Travel',
    'Narrative general': 'Narrative',
    'Interactive discussion general': 'Discussion',
    'Fiction': 'Fiction',
    'FAQs': 'FAQs',
    'Legal terms / conditions': 'Legal',
    'Informational Description general': 'D-Informational',
    'Course materials': 'Course',
    'Interviews': 'Interviews',
    'Report': 'Report',
    'Formal speeches': 'Speeches',
    'Recipes': 'Recipes',
    'Informational Persuasion general': 'Info-Persuasion',
}


def load_data(fn):
    texts, classes = [], []
    with open(fn) as f:
        data = json.load(f)
    for doc_id, doc_data in data.items():
        try:
            text = doc_data['text']
        except KeyError:
            warning('no text for {}, skipping'.format(doc_id))
            continue
        class_ = doc_data['register-1']
        try:
            class_ = CLASS_MAP[class_]
        except KeyError:
            warning('no mapping for class {}'.format(class_))
        texts.append(text)
        classes.append(class_)
    return texts, classes


train_texts, train_classes = load_data('../data/annotations-train.json')
devel_texts, devel_classes = load_data('../data/annotations-dev.json')

In [24]:
from collections import Counter


MIN_EXAMPLES = 25    # filter classes with fewer


class_count = Counter()
for c in train_classes:
    class_count[c] += 1
target_class = set(c for c, v in class_count.items() if v >= MIN_EXAMPLES)


def filter_by_class(texts, classes, targets):
    filtered_texts, filtered_classes = [], []
    for t, c in zip(texts, classes):
        if c in targets:
            filtered_texts.append(t)
            filtered_classes.append(c)
    return filtered_texts, filtered_classes


train_texts, train_classes = filter_by_class(train_texts, train_classes, target_class)
devel_texts, devel_classes = filter_by_class(devel_texts, devel_classes, target_class)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC


vectorizer = TfidfVectorizer(analyzer='word', lowercase=False, ngram_range=(1,3))
vectorizer.fit(train_texts)

train_X = vectorizer.transform(train_texts)
devel_X = vectorizer.transform(devel_texts)

classifier = LinearSVC(C=1.0)
classifier.fit(train_X, train_classes)

classifier.score(devel_X, devel_classes)

0.6116504854368932

In [26]:
import eli5


eli5.show_weights(classifier, vec=vectorizer, top=(100,100))

In [27]:
import numpy as np

from sklearn.metrics import confusion_matrix
from pandas import DataFrame


pred_Y = classifier.predict(devel_X)
cm = confusion_matrix(devel_classes, pred_Y)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]    # normalize
df = DataFrame(cm * 100, index=classifier.classes_, columns=classifier.classes_)
df.round(2)

,A-Encyclopedia,B-Community,B-Personal,B-Personal-Opinion,D-Sell,D-Thing,Editorials,Forums,How-to,MT/Gen,News,Religious,Reviews,Sports
A-Encyclopedia,20.0,0.0,0.00,0.00,20.00,0.00,0.0,0.00,0.0,0.00,20.00,40.00,0.00,0.0
B-Community,0.0,0.0,37.50,0.00,0.00,0.00,0.0,0.00,0.0,37.50,25.00,0.00,0.00,0.0
B-Personal,0.0,0.0,90.00,0.00,0.00,0.00,0.0,0.00,0.0,10.00,0.00,0.00,0.00,0.0
B-Personal-Opinion,0.0,0.0,20.00,50.00,0.00,0.00,0.0,0.00,0.0,20.00,10.00,0.00,0.00,0.0
D-Sell,0.0,0.0,0.00,3.03,81.82,0.00,0.0,0.00,0.0,9.09,6.06,0.00,0.00,0.0
D-Thing,0.0,0.0,0.00,0.00,60.00,6.67,0.0,0.00,0.0,20.00,6.67,6.67,0.00,0.0
Editorials,0.0,0.0,25.00,0.00,0.00,0.00,0.0,0.00,0.0,50.00,25.00,0.00,0.00,0.0
Forums,0.0,0.0,77.78,0.00,0.00,0.00,0.0,0.00,0.0,0.00,11.11,11.11,0.00,0.0
How-to,0.0,0.0,8.33,0.00,16.67,0.00,0.0,0.00,25.0,41.67,8.33,0.00,0.00,0.0
MT/Gen,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,100.00,0.00,0.00,0.00,0.0
